# Afterburner Data Collector

## 1. Environment Setup

### 1-1. Install Packages

In [105]:
import time
import json

import pandas as pd

### 1-2. Selenium Setup

In [30]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [40]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
wd = webdriver.Chrome('chromedriver', options=chrome_options)

### 1-3. Global Variables

In [ ]:
NUM_SCROLL = 10
SHORT_WAIT_TIME = 2
MEDIUM_WAIT_TIME = 5
LONG_WAIT_TIME = 10

## 2. Data Collection

### 2-1. Instagram Login

In [102]:
with open("keys.json", "r") as f:
    keys = json.load(f)

In [60]:
wd.get('https://www.instagram.com')
time.sleep(MEDIUM_WAIT_TIME)

In [61]:
username = wd.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input')
password = wd.find_element_by_css_selector('#loginForm > div > div:nth-child(2) > div > label > input')
login = wd.find_element_by_css_selector('#loginForm > div > div:nth-child(3) > button')

In [62]:
username.send_keys(keys['id'])
time.sleep(SHORT_WAIT_TIME)
password.send_keys(keys['pw'])
time.sleep(SHORT_WAIT_TIME)
login.click()
time.sleep(SHORT_WAIT_TIME)

In [67]:
wd.find_element_by_css_selector('#react-root > section > div > div > div.GA2q- > form > span > button').click()
time.sleep(SHORT_WAIT_TIME)

In [69]:
wd.find_element_by_css_selector('input').click()
time.sleep(SHORT_WAIT_TIME)

code = input('Keys:')

security.send_keys(code)
time.sleep(SHORT_WAIT_TIME)

Keys:271094


In [71]:
submit = wd.find_element_by_css_selector('#react-root > section > div > div > div.GA2q- > form > span > button')
submit.click()
time.sleep(SHORT_WAIT_TIME)

### 2-2. Retrieve Post List

In [74]:
wd.get('https://www.instagram.com/explore/tags/공스타그램/')

In [83]:
for i in range(NUM_SCROLL):
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(LONG_WAIT_TIME)

wd.get('https://www.instagram.com')
time.sleep(LONG_WAIT_TIME)

In [94]:
posts = wd.find_elements_by_css_selector('#react-root > section > main > article > div:nth-child(3) > div > div > div > a')
print(len(posts))

In [97]:
for post in posts:
    wd.get(post.get_attribute('href'))

    wd.find_elements_by_css_selector('#react-root > section > main > div > div.ltEKP > article > div.eo2As > div.EtaWk > ul > div > li > div > div > div.C4VMK > span')

    time.sleep(LONG_WAIT_TIME)

https://www.instagram.com/p/CTJQBaShlaj/
https://www.instagram.com/p/CTJP-DslAWW/
https://www.instagram.com/p/CTJP7t5Fqfp/
https://www.instagram.com/p/CTJP7PZhvPI/
https://www.instagram.com/p/CTJP5IenjHK/
https://www.instagram.com/p/CTJPzLAp7Dc/
https://www.instagram.com/p/CTJPj4pBVg3/
https://www.instagram.com/p/CTJPmy1hshp/
https://www.instagram.com/p/CTJPmYKL2wm/
https://www.instagram.com/p/CTJPmRhhCo4/
https://www.instagram.com/p/CTJPdcalbBL/
https://www.instagram.com/p/CTJPNnYBAmc/
https://www.instagram.com/p/CTJPaB5nAPw/
https://www.instagram.com/p/CTJOz-phVc5/
https://www.instagram.com/p/CTJEkKwHY6B/
https://www.instagram.com/p/CTJCZKRpvJD/
https://www.instagram.com/p/CTIPlTuhd1W/
https://www.instagram.com/p/CTGnmR5AhU3/
https://www.instagram.com/p/CTJPcYph4BF/
https://www.instagram.com/p/CTJPYhnBoJ7/
https://www.instagram.com/p/CTJPUU3Bs-6/
https://www.instagram.com/p/CTJPRnzh4bF/
https://www.instagram.com/p/CTJPRovhD7n/
https://www.instagram.com/p/CTJPRbghs1m/
https://www.inst

In [96]:
wd.save_screenshot('current.png')

True